In [1]:
import numpy as np
import healpy as hp
import json
from astropy.table import Table, vstack, join
from astropy.io import fits
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '/global/homes/h/hrincon/python_tools')
import VoidVolume as vol
import VoidOverlap as vo
import VoidCatalog as vc
import VoidSlicePlots as vsp

from vast.voidfinder.postprocessing import mknum

<frozen importlib._bootstrap>:241: RuntimeWarning: vast.voidfinder._voidfinder_cython_find_next.SpatialMap size changed, may indicate binary incompatibility. Expected 1848 from C header, got 2272 from PyObject


# Read in Void Catalogs

In [2]:
#DESI
#------
redtext = lambda x: f"\x1b[31m{x}\x1b[0m"

for desi_mag, sdss_mag in zip([-19.89, -20.06], [-19.94, -20.11]):

    print("\nMagnitudes:", desi_mag, sdss_mag,'\n')

    desi_mag = np.abs(desi_mag)
    sdss_mag = np.abs(sdss_mag)
    
    iron_smoothed_ngc_path = f'../galaxy_catalog/iron_smoothed_ngc.fits'
    iron_smoothed_sgc_path = f'../galaxy_catalog/iron_smoothed_sgc.fits'

    #VoidFinder
    voids_ngc_path = f'../voids/data/minus{desi_mag}/DESIVAST_NGC_VoidFinder_Output.fits'
    voids_sgc_path = f'../voids/data/minus{desi_mag}/DESIVAST_SGC_VoidFinder_Output.fits'

    vfc = vc.VoidFinderCatalogStacked(['NGC','SGC'],[voids_ngc_path,voids_sgc_path])
    vfc.add_galaxies([iron_smoothed_ngc_path,iron_smoothed_sgc_path])  

    #VIDE
    voids_ngc_path = f'../voids/data/minus{desi_mag}/DESIVAST_NGC_V2_VIDE_Output.fits'
    voids_sgc_path = f'../voids/data/minus{desi_mag}/DESIVAST_SGC_V2_VIDE_Output.fits'

    v2v = vc.V2CatalogStacked(['NGC','SGC'],[voids_ngc_path,voids_sgc_path])
    v2v.add_galaxies([iron_smoothed_ngc_path,iron_smoothed_sgc_path]) 

    #REVOLVER
    voids_ngc_path = f'../voids/data/minus{desi_mag}/DESIVAST_NGC_V2_REVOLVER_Output.fits'
    voids_sgc_path = f'../voids/data/minus{desi_mag}/DESIVAST_SGC_V2_REVOLVER_Output.fits'

    v2r = vc.V2CatalogStacked(['NGC','SGC'],[voids_ngc_path,voids_sgc_path])
    v2r.add_galaxies([iron_smoothed_ngc_path,iron_smoothed_sgc_path]) 

    #SDSS
    #------
    nsa_path= f'../galaxy_catalog/SDSS_galaxies/nsa_k1.fits'

    #VoidFinder
    voids_ngc_path = f'../voids/data/minus{sdss_mag}/SDSSK1_VoidFinder_Output.fits'

    vfc_sdss = vc.VoidFinderCatalog(voids_ngc_path)
    vfc_sdss.add_galaxies(nsa_path)  

    #VIDE
    voids_ngc_path = f'../voids/data/minus{sdss_mag}/SDSSK1_V2_VIDE_Output.fits'

    v2v_sdss = vc.V2Catalog(voids_ngc_path)
    v2v_sdss.add_galaxies(nsa_path)  

    #REVOLVER
    voids_ngc_path = f'../voids/data/minus{sdss_mag}/SDSSK1_V2_REVOLVER_Output.fits'

    v2r_sdss = vc.V2Catalog(voids_ngc_path)
    v2r_sdss.add_galaxies(nsa_path)  
    
    # copy over masks
    
    v2r['NGC'].mask = vfc['NGC'].mask
    v2r['NGC'].mask_info = vfc['NGC'].mask_info
    v2v['NGC'].mask = vfc['NGC'].mask
    v2v['NGC'].mask_info = vfc['NGC'].mask_info

    v2r['SGC'].mask = vfc['SGC'].mask
    v2r['SGC'].mask_info = vfc['SGC'].mask_info
    v2v['SGC'].mask = vfc['SGC'].mask
    v2v['SGC'].mask_info = vfc['SGC'].mask_info

    v2r_sdss.mask = vfc_sdss.mask
    v2r_sdss.mask_info = vfc_sdss.mask_info
    v2v_sdss.mask = vfc_sdss.mask
    v2v_sdss.mask_info = vfc_sdss.mask_info
    
    print('\n'+redtext("DESI VoidFinder"),'\n')
    vfc.void_stats(report_individual=False)
    print('\n'+redtext("DESI VIDE"),'\n')
    v2v.void_stats(report_individual=False)
    print('\n'+redtext("DESI REVOVLER"),'\n')
    v2r.void_stats(report_individual=False)
    print('\n'+redtext("SDSS VoidFinder"),'\n')
    vfc_sdss.void_stats()
    print('\n'+redtext("SDSS VIDE"),'\n')
    v2v_sdss.void_stats()
    print('\n'+redtext("SDSS REVOLVER"),'\n')
    v2r_sdss.void_stats()


Magnitudes: -19.89 -19.94 


DESI VoidFinder 

3736 voids
2205 edge voids
58 near-edge voids
1473 interior voids
Mean Reff (V. Fid): 16.41 +/- 0.0488 Mpc/h
Median Reff (V. Fid): 15.68 +/- 0.0612 Mpc/h
Maximum Reff (V. Fid): 30.92 Mpc/h

DESI VIDE 

1581 voids
Mean Reff (V. Fid): 20.45 +/- 0.203 Mpc/h
Median Reff (V. Fid): 18.33 +/- 0.255 Mpc/h
Maximum Reff (V. Fid): 53.67 Mpc/h

DESI REVOVLER 

2143 voids
Mean Reff (V. Fid): 19.95 +/- 0.139 Mpc/h
Median Reff (V. Fid): 19.13 +/- 0.174 Mpc/h
Maximum Reff (V. Fid): 39.63 Mpc/h

SDSS VoidFinder 

1163 voids
339 edge voids
20 near-edge voids
804 interior voids
6 edge voids (V. Fid)
0 near-edge voids (V. Fid)
677 interior voids (V. Fid)
Mean Reff (V. Fid): 16.48 +/- 0.0893 Mpc/h
Median Reff (V. Fid): 15.74 +/- 0.112 Mpc/h
Maximum Reff (V. Fid): 28.98 Mpc/h

SDSS VIDE 

532 voids
Mean Reff (V. Fid): 19.84 +/- 0.326 Mpc/h
Median Reff (V. Fid): 18.24 +/- 0.408 Mpc/h
Maximum Reff (V. Fid): 51.2 Mpc/h

SDSS REVOLVER 

715 voids
Mean Reff (V. Fid

In [14]:
np.median(
    np.array(
        vstack(
            [vfc['NGC'].maximals[vfc['NGC'].maximals['r']<332], 
             vfc['SGC'].maximals[vfc['SGC'].maximals['r']<332]]
        )['r_eff']
    ).astype(float)
)

14.703377818524995

In [15]:
np.median(
    np.array(
        vstack(
            [v2v['NGC'].voids[v2v['NGC'].voids['redshift']<.114], 
             v2v['SGC'].voids[v2v['SGC'].voids['redshift']<.114]]
        )['radius']
    ).astype(float)
)

14.419070517469825

In [16]:
np.median(
    np.array(
        vstack(
            [v2r['NGC'].voids[v2r['NGC'].voids['redshift']<.114], 
             v2r['SGC'].voids[v2r['SGC'].voids['redshift']<.114]]
        )['radius']
    ).astype(float)
)

14.57734270426593

In [17]:
masks_path = f'../galaxy_catalog/mask/alt_masks/masks.fits'

masks = fits.open(masks_path)